<a href="https://colab.research.google.com/github/DevanshParmar/Data-Science-Summer-Camp-2021/blob/main/Logistic%20Regression%20Model%20on%20Titanic%20Survival%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Logistic Regression Model on Titanic Survival Dataset**
This is an implementation of the Logistic Regression Model, one of the most basic machine learning model on the Titanic survival dataset. 

#### **Uploads**
Setting up libraries and uploading dataset files.


In [ ]:
import numpy as np
import pandas as pd
import zipfile
from google.colab import drive
import os

In [ ]:
drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
%cd /content/gdrive/My Drive/Kaggle
!kaggle competitions download -c titanic
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Kaggle
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv  kaggle.json  test.csv  train.csv


In [ ]:
!unzip \*.zip  && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


#### **Data Preprocessing**
Constructing a dataframe and making necessary changes, such as deleting PassId and converting Sex to male=1, female=0 objective case.

In [ ]:
print("Input Dataframes:")
data = pd.read_csv('/content/gdrive/MyDrive/Kaggle/train.csv')
col_names1 = ['PassId', 'Survived', 'PClass', 'Name', 'Sex', 'Age', 'SibSp', 'ParCh', 'Ticket', 'Fare', 'Cabin', 'Embarked']
data.columns = col_names1
print(data.head())
print(" ")

test = pd.read_csv('/content/gdrive/MyDrive/Kaggle/test.csv')
col_names2 = ['PassId', 'PClass', 'Name', 'Sex', 'Age', 'SibSp', 'ParCh', 'Ticket', 'Fare', 'Cabin', 'Embarked']
test.columns = col_names2
print(test.head())
print(" ")

test_survived = pd.read_csv('/content/gdrive/MyDrive/Kaggle/gender_submission.csv')
test_survived.columns = ['PassId', 'Survived']
print(test_survived.head())
print(" ")

print("Output Dataframes:")
data.pop("PassId")
data.pop("Name")
data.pop("Ticket")
data.pop("Cabin")
data.pop("Embarked")
ifmale = pd.get_dummies(data['Sex'], drop_first = True)
data = pd.concat([data, ifmale], axis = 1)
data.pop("Sex")
print(data.head())
print(" ")

test.pop("PassId")
test.pop("Name")
test.pop("Ticket")
test.pop("Cabin")
test.pop("Embarked")
ifmale = pd.get_dummies(test['Sex'], drop_first = True)
test = pd.concat([test, ifmale], axis = 1)
test.pop("Sex")
test_survived.pop("PassId")
test['Survived'] = test_survived.values
print(test.head())
print(" ")

Input Dataframes:
   PassId  Survived  PClass  ...     Fare Cabin  Embarked
0       1         0       3  ...   7.2500   NaN         S
1       2         1       1  ...  71.2833   C85         C
2       3         1       3  ...   7.9250   NaN         S
3       4         1       1  ...  53.1000  C123         S
4       5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]
 
   PassId  PClass  ... Cabin Embarked
0     892       3  ...   NaN        Q
1     893       3  ...   NaN        S
2     894       2  ...   NaN        Q
3     895       3  ...   NaN        S
4     896       3  ...   NaN        S

[5 rows x 11 columns]
 
   PassId  Survived
0     892         0
1     893         1
2     894         0
3     895         0
4     896         1
 
Output Dataframes:
   Survived  PClass   Age  SibSp  ParCh     Fare  male
0         0       3  22.0      1      0   7.2500     1
1         1       1  38.0      1      0  71.2833     0
2         1       3  26.0      0      0   7.9250  

Filling NaN Age values with median age

In [ ]:
data['Age'] = data['Age'].fillna(data['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
print(data)
print(" ")
print(test)

     Survived  PClass   Age  SibSp  ParCh     Fare  male
0           0       3  22.0      1      0   7.2500     1
1           1       1  38.0      1      0  71.2833     0
2           1       3  26.0      0      0   7.9250     0
3           1       1  35.0      1      0  53.1000     0
4           0       3  35.0      0      0   8.0500     1
..        ...     ...   ...    ...    ...      ...   ...
886         0       2  27.0      0      0  13.0000     1
887         1       1  19.0      0      0  30.0000     0
888         0       3  28.0      1      2  23.4500     0
889         1       1  26.0      0      0  30.0000     1
890         0       3  32.0      0      0   7.7500     1

[891 rows x 7 columns]
 
     PClass   Age  SibSp  ParCh      Fare  male  Survived
0         3  34.5      0      0    7.8292     1         0
1         3  47.0      1      0    7.0000     0         1
2         2  62.0      0      0    9.6875     1         0
3         3  27.0      0      0    8.6625     1         0


Sending dataframe to NumPy

In [ ]:
input_train = data[['PClass', 'Age', 'SibSp', 'ParCh', 'Fare', 'male']].to_numpy()
output_train = data['Survived'].to_numpy()
print(input_train.shape)
print(input_test.shape)

(891, 6)
(418, 6)


#### **Modeling**
In the next blocks, we define, train and optimize our model.
1. The first block defines the sigmoid function, necessary for logistic regression.
2. The second block defines the optimization function.
3. The third block puts up the inital parameters as 0.
4. The fourth block runs the training sessions of the model, and finallly outputs the trained theta parameters of the model.


In [ ]:
def g(z):
    return 1/(1+np.exp(-z))

In [ ]:
def optimize(x, y, learning_rate, N_iterations, parameters):
    size = x.shape[0] #620
    weights = parameters["weights"]                                             #theta1-6
    bias = parameters["bias"]                                                   #theta0
    for i in range(N_iterations):
        h = g(bias + np.dot(x, weights))                                        #h becomes the hypothesis function
        loss = -1/size * np.sum(y * np.log(h)) + (1 - y) * np.log(1-h)          #log-likelihood
        del_weights = 1/size * np.dot(x.T, (h-y))                               #change in weights
        del_bias = 1/size * np.sum(h-y)                                         #change in bias
        weights = weights - learning_rate * del_weights                         #learning
        bias = bias - learning_rate * del_bias                                  #learning
    
    parameters["weights"] = weights
    parameters["bias"] = bias
    return parameters

In [ ]:
initial_parameters = {}
initial_parameters["weights"] = np.zeros(input_train.shape[1])
initial_parameters["bias"] = 0

In [ ]:
def train(x, y, lr, N_it):
    return optimize(x, y, lr, N_it, initial_parameters)

theta = train(input_train, output_train, lr = 0.005, N_it = 5000)
print(theta['weights'])

[-0.10967463 -0.01136579 -0.29063817  0.02180507  0.00997821 -1.51893593]


#### **Predictions and Accuracy**
In the next two blocks, we have measured the various statistical parameters of our model, such as accuracy, loss, F1 score, sensitivity and precision.

In [ ]:
def stats(dataset):
    z = []
    for i in range(dataset.shape[0]):
        p  = 0.0
        p += theta["bias"]
        p += theta["weights"][0] * dataset['PClass'][i]
        p += theta["weights"][1] * dataset['Age'][i]
        p += theta["weights"][2] * dataset['SibSp'][i]
        p += theta["weights"][3] * dataset['ParCh'][i]
        p += theta["weights"][4] * dataset['Fare'][i]
        p += theta["weights"][5] * dataset['male'][i]
        z.insert(len(z), p)
    #                              
    sigmoids = []
    for val in z:
        sigmoids.insert(len(sigmoids), g(val))
    #                              
    predictions = []
    for p in sigmoids:
        if p >= 0.5:
            predictions.insert(len(predictions), 1)
        else:
            predictions.insert(len(predictions), 0)
    prediction = np.array(predictions)
    survive_data = np.array(dataset['Survived'])
    #                              
    loss = 0
    f_neg = 0
    f_pos = 0 
    t_neg = 0
    t_pos = 0
    #                              
    for i, j in zip(prediction, survive_data):
        if i == 1 and j == 1:
            t_pos+=1
        elif i == 1 and j == 0:
            f_pos+=1
            loss+=1
        elif i==0 and j == 1:
            f_neg+=1
            loss+=1
        else:
            t_neg+=1
    #                              
    rec = t_pos / (t_pos + f_neg)
    prc = t_pos / (t_pos + f_pos)
    acc = (t_pos + t_neg) / (t_pos + t_neg + f_pos + f_neg)
    f1s = 2 * prc * rec / (prc + rec)
    #                              
    print('   Accuracy is {:.2f}%'.format(100*acc))
    print('       Loss is',loss)
    print('   F1 Score is {:.4f}'.format(f1s))
    print('Sensitivity is {:.4f}'.format(rec))
    print('  Precision is {:.4f}'.format(prc))

In [ ]:
print("Statistics for Training dataset are:")
print(" ")
stats(data)
print(" ")
print(" ")
print("Statistics for Test dataset are:")
print(" ")
stats(test)

Statistics for Training dataset are:
 
   Accuracy is 73.06%
       Loss is 240
   F1 Score is 0.4958
Sensitivity is 0.3450
  Precision is 0.8806
 
 
Statistics for Test dataset are:
 
   Accuracy is 77.75%
       Loss is 93
   F1 Score is 0.5830
Sensitivity is 0.4276
  Precision is 0.9155


#### **References**

1. These lecture notes were extremely helpful in understanding the mathematics of the logistic regression: https://see.stanford.edu/materials/aimlcs229/cs229-notes1.pdf
2. Many Towards Data Science (TDS) articles were helpful, especially: www.towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
3. Another TDS blog was a good help: www.towardsdatascience.com/optimization-loss-function-under-the-hood-part-ii-d20a239cde11
4. This Exsilio blog was greatly helpful in visualing the final statistics of the model: www.blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/